In [ ]:
import numpy as np
!pip install pyquaternion
from pyquaternion import Quaternion
from scipy.spatial.transform import Rotation
from scipy.integrate import odeint
import pandas as pd


In [ ]:
def hat_map(vector):
    """
    Convert a vector into a skew-symmetric matrix.
    """
    matrix = np.array([[0, -vector[2], vector[1]],
              [vector[2], 0, -vector[0]],
              [-vector[1], vector[0], 0]
    ], dtype=np.float64)
    return matrix


In [ ]:
def vee_map(matrix):
    """
    Convert a skew-symmetric matrix to a vector.
    """
    vector = [-matrix[1][2], matrix[0][2], -matrix[0][1]]
    return np.array(vector)


In [ ]:
def quad_dynamics(t, x, u, params):
    mass = params["mass"]
    J = params["J"]
    g = params["g"]
    e3 = np.array([0, 0, 1])

    # States extraction
    v = x[3:6].reshape(3,1)
    R = x[6:15].reshape(3, 3, order='F')
    omega = x[15:18].reshape(3,1)
    # print('v', v)
    # print('R', R)
    # print('omega', omega)

    # Convert rotation matrix to quaternion using scipy's Rotation
    rotation = Rotation.from_matrix(R.T)
    q = rotation.as_quat()
    q = np.array([q[3], q[0], q[1], q[2]])

    G = np.array([
        [-q[1], -q[2], -q[3]],
        [q[0], -q[3], q[2]],
        [q[3], q[0], -q[1]],
        [-q[2], q[1], q[0]]
    ])


    # Control inputs extraction

    F = u[0]
    M = u[1:4].reshape(3,1)

    # Dynamics
    a = np.dot(R, (1/mass * F * e3)) - g * e3
    dR = np.dot(R, hat_map(omega))
    # print('a', a)
    # print('dR', dR)

    dq = 0.5 * np.dot(G, omega)

    quat_err = np.sum(q**2) - 1

    quat_err_grad = 2 * q.reshape(4, 1)

    dq = dq - quat_err * quat_err_grad
    omega_dot = np.linalg.solve(J, (M - np.dot(hat_map(omega), np.dot(J, omega))))



    # Constructing x_dot
    x_dot = np.concatenate([v.flatten(), a, dR.flatten(order='F'), omega_dot.flatten(), dq.flatten()])
    # print('dR', dR)

    return x_dot

In [ ]:
#Test
x1_test = np.array([
    -0.00323963905235902,
    0.0203285842770193,
    0.00903262252436420,
    -0.317273629030219,
    0.491600194953207,
    0.00834001035537005,
    -0.688112422600223,
    -0.184687847777963,
    -0.701662042388847,
    0.225973010603088,
    -0.973516623098761,
    0.0346949130395443,
    -0.689546735902591,
    -0.134691252639382,
    0.711625072756379,
    -20.9563597787239,
    -40.0234806428743,
    48.0475926188807,
    1.81555746602065,
    -0.915327995315078,
    0.116291026238883,
    0.496482964039017
])

# Using previously defined control input and parameters
F = 4.9
M = [0.1, -0.2, 0.15]
u_test_py = np.concatenate([[F], M])
params_test_py = {
    "mass": 1.2,
    "J": np.array([[0.04, 0, 0], [0, 0.045, 0], [0, 0, 0.03]]),
    "g": 9.81
}

# # Execute the quad_dynamics Python function with the test data
x_dot_python_result = quad_dynamics(0, x1_test, u_test_py, params_test_py)


MATLAB_OUTPUT = [
   -0.3173,
    0.4916,
    0.0083,
   -2.8156,
   -0.5500,
   -6.9042,
  -16.7406,
  -52.1659,
   30.1487,
   47.5125,
   11.6964,
   18.8001,
   32.2762,
  -13.0095,
   28.8100,
 -718.6370,
  219.3117,
 -134.7911,
  -26.5749,
   16.5517,
   -2.7609,
    9.9798
]
print(x_dot_python_result)
for i, (python_val, matlab_val) in enumerate(zip(x_dot_python_result, MATLAB_OUTPUT)):
    assert abs(python_val - matlab_val) < 1e-3, f"Discrepancy in index {i}: Python={python_val}, MATLAB={matlab_val}"

print("All values match with MATLAB for quad_dynamics!")

[-3.17273629e-01  4.91600195e-01  8.34001036e-03 -2.81564917e+00
 -5.49989282e-01 -6.90419762e+00 -1.67406013e+01 -5.21659429e+01
  3.01487194e+01  4.75125348e+01  1.16964448e+01  1.88001009e+01
  3.22762259e+01 -1.30095141e+01  2.88100363e+01 -7.18636960e+02
  2.19311697e+02 -1.34791077e+02 -2.65748726e+01  1.65518293e+01
 -2.76088409e+00  9.97974975e+00]
All values match with MATLAB for quad_dynamics!


<ipython-input-2-abdd456dfe88>:5: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  matrix = np.array([[0, -vector[2], vector[1]],


In [ ]:
def compute_rmse(X, X_ref):
    x_ref = X_ref[0:3, :]
    dx_ref = X_ref[3:6, :]
    wb_ref = X_ref[15:18, :]
    x = X[0:3, :]
    dx = X[3:6, :]
    wb = X[15:18, :]

    theta_err = np.zeros((3, X.shape[1]))
    angle_err = np.zeros((1, X.shape[1]))

    for i in range(X.shape[1]):
        q_true = Quaternion(X_ref[18:22, i])
        q1 = Quaternion(X[18:22, i])

        R_true = q_true.rotation_matrix
        R = q1.rotation_matrix

        theta_err[:, i] = 0.5 * vee_map(R_true.T @ R - R.T @ R_true)

        q_est = q1.conjugate.normalised
        q_temp = q_est * q_true

        angle_err[:, i] = 2 * np.arctan2(np.linalg.norm([q_temp.x, q_temp.y, q_temp.z]), q_temp.w)

    x_error = x_ref.flatten() - x.flatten()
    RMSE_x = np.sqrt(np.mean(x_error**2))

    dx_error = dx_ref.flatten() - dx.flatten()
    RMSE_dx = np.sqrt(np.mean(dx_error**2))

    ang_error = angle_err
    RMSE_quat_angle = np.sqrt(np.mean(ang_error**2))

    theta_error = theta_err.flatten()
    RMSE_theta = np.sqrt(np.mean(theta_error**2))

    wb_error = wb_ref.flatten() - wb.flatten()
    RMSE_wb = np.sqrt(np.mean(wb_error**2))

    return RMSE_x, RMSE_dx, RMSE_quat_angle, RMSE_theta, RMSE_wb

In [ ]:
#Test

# Define the x2 values
x2 = np.array([
    -0.0164284515357175,
    0.0135039823114394,
    0.00384578516581931,
    -0.267477161555976,
    0.531354730089236,
    -0.142354964676809,
    -0.810260922329120,
    0.427790887673259,
    0.400606198206329,
    0.0106633511269538,
    -0.672663882499665,
    0.739861189667185,
    0.585972466939531,
    0.603764501652060,
    0.540456569508744,
    -50.4611106830803,
    -4.43369350556665,
    -42.0963286807543,
    1.82725780608126,
    -0.384890404887259,
    -0.887675995095501,
    -0.404902927765349
]).reshape(-1, 1)

# Perturb x2 using sin and cos functions to create X_ref
X_ref = x2 + np.sin(x2) + np.cos(x2)

# Call the compute_rmse function
rmse_values = compute_rmse(x2, X_ref)

# Display the results
print("RMSE values:")
print(rmse_values)

MATLAB_RMSE = {
    'x': 1.0003,
    'dx': 1.0137,
    'quat_angle': 0.6968,
    'theta': 0.3705,
    'wb': 0.7072
}

# Extract RMSE values
RMSE_x, RMSE_dx, RMSE_quat_angle, RMSE_theta, RMSE_wb = rmse_values

# Add an assert block
assert abs(RMSE_x - MATLAB_RMSE['x']) < 1e-4, f"Discrepancy in RMSE_x: {RMSE_x}"
assert abs(RMSE_dx - MATLAB_RMSE['dx']) < 1e-4, f"Discrepancy in RMSE_dx: {RMSE_dx}"
assert abs(RMSE_quat_angle - MATLAB_RMSE['quat_angle']) < 1e-4, f"Discrepancy in RMSE_quat_angle: {RMSE_quat_angle}"
assert abs(RMSE_theta - MATLAB_RMSE['theta']) < 1e-4, f"Discrepancy in RMSE_theta: {RMSE_theta}"
assert abs(RMSE_wb - MATLAB_RMSE['wb']) < 1e-4, f"Discrepancy in RMSE_wb: {RMSE_wb}"

print("All RMSE values match with MATLAB!")




RMSE values:
(1.0003074834539418, 1.0137383034770706, 0.6967555628037899, 0.37050461551731617, 0.7071664126990628)
All RMSE values match with MATLAB!


In [ ]:
def construct_basis(X, n_basis):
    # Extract states
    R = X[6:15].reshape(3, 3, order='F')
    wb = X[15:18].reshape(3, 1)
    Q = X[18:22].reshape(4, 1)
    wb_hat = hat_map(wb)

    # Build observables
    basis = np.zeros((9 * n_basis, 1))
    Z = R
    for i in range(n_basis):
        Z = np.dot(Z, wb_hat)
        basis[i*9:(i+1)*9, :] = Z.flatten(order='F').reshape(9, 1)

    # Concatenate results
    basis_result = np.concatenate([R.flatten(order='F').reshape(9, 1), wb, Q, basis])

    return basis_result

In [ ]:
#Test
n_basis = 5

# Call the construct_basis function
basis_result = construct_basis(x2, n_basis)

# # Display the results
# print("Basis Result:")
# print(basis_result)

MATLAB_OUTPUT = [
    -0.810260922329120, 0.427790887673259, 0.400606198206329, 0.0106633511269538, -0.672663882499665,
    0.739861189667185, 0.585972466939531, 0.603764501652060, 0.540456569508744, -50.4611106830803,
    -4.43369350556665, -42.0963286807543, 1.82725780608126, -0.384890404887259, -0.887675995095501,
    -0.404902927765349, 2.14913438723213, 30.9935866392446, -28.7492210360920, -63.6778316150111,
    -12.4582015302542, -10.4079885821376, 4.13053313061482, -35.8400603077437, 35.5580522826342,
    2698.91644725897, 365.540703763099, 595.791613735315, -117.960621940050, 3113.24546023840,
    -3004.53546995974, -3222.78271235888, -766.070750126916, -397.733429233658, -9323.12166906352,
    -134452.727067704, 124716.484554270, 276239.669001567, 54044.7025250303, 45150.7101919596,
    -17918.5923242409, 155477.128308393, -154253.754309369, -11708121.4473696, -1585745.62690220,
    -2584593.00510094, 511722.876446954, -13505514.7709231, 13033921.8951905, 13980696.3765413,
    3323277.89875551, 1725400.31681078, 40444468.3275981, 583266984.436468, -541030363.920073,
    -1198350395.96585, -234450362.993880, -195867493.007918, 77732326.7310423, -674472565.600050,
    669165468.609081
]

# Check the difference between the MATLAB output and the Python function result
for i, (python_val, matlab_val) in enumerate(zip(basis_result, MATLAB_OUTPUT)):
    assert abs(python_val - matlab_val) < 1e-4, f"Discrepancy in index {i}: Python={python_val}, MATLAB={matlab_val}"

print("All values match with MATLAB for construct_basis!")



All values match with MATLAB for construct_basis!


<ipython-input-2-abdd456dfe88>:5: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  matrix = np.array([[0, -vector[2], vector[1]],


In [ ]:
def EDMD(Z1, Z2, U):
    Z1_aug = np.vstack((Z1, U))

    m = Z1.shape[1]
    Y = np.dot(Z2, Z1_aug.T) / m
    G = np.dot(Z1_aug, Z1_aug.T) / m

    K = np.dot(Y, np.linalg.pinv(G))
    A = K[:, :Z1.shape[0]]
    B = K[:, Z1.shape[0]:]

    return A, B

In [ ]:
#Test:

from google.colab import files
uploaded = files.upload()


Saving B_test.csv to B_test.csv
Saving A_test.csv to A_test.csv
Saving U_Train.csv to U_Train.csv
Saving Z2_Train.csv to Z2_Train.csv
Saving Z1_Train.csv to Z1_Train.csv
Saving train_dat_wquat.mat to train_dat_wquat.mat
Saving val_dat_wquat.mat to val_dat_wquat.mat


In [ ]:

def compare_csvs(file1, file2):
    # Read the CSV files into DataFrames
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Check if the shapes of the two DataFrames are the same
    if df1.shape != df2.shape:
        return False

    # Check if all values in the DataFrames are the same
    if not df1.equals(df2):
        return False

    return True

In [ ]:

U_df = pd.read_csv('U_train.csv', header=None)
U = U_df.values


Z1_df = pd.read_csv('Z1_Train.csv', header=None)
Z1 = Z1_df.values



Z2_df = pd.read_csv('Z2_Train.csv', header=None)
Z2 = Z2_df.values

A_df = pd.read_csv('A_test.csv', header=None)
A = A_df.values

B_df = pd.read_csv('B_test.csv', header=None)
B = B_df.values

# Call the EDMD function
A_result, B_result = EDMD(Z1, Z2, U)

Adf = pd.DataFrame(A_result)
Bdf = pd.DataFrame(B_result)


# Writing data to a CSV file
Adf.to_csv('A_result.csv', index=False)
Adf.to_csv('B_result.csv', index=False)

if compare_csvs('A_result.csv', 'A_test.csv'):
    print("The two CSV files are identical.")
else:
    print("The two CSV files are different.")
# print("A Result:\n", A_result)
# print('A', A)
DISCREPANCY_THRESHOLD = 1e-3

# # List to collect discrepancies
# discrepancies = []

# for i, (python_val, matlab_val) in enumerate(zip(x_dot_python_result, MATLAB_OUTPUT)):
#     difference = abs(python_val - matlab_val)
#     if difference > DISCREPANCY_THRESHOLD:
#         discrepancies.append((i, python_val, matlab_val, difference))

# # If discrepancies found, print them
# if discrepancies:
#     for idx, py_val, mat_val, diff in discrepancies:
#         print(f"Discrepancy in index {idx}: Python={py_val}, MATLAB={mat_val}, Difference={diff}")
#     raise AssertionError("Discrepancies found between Python and MATLAB outputs.")
# else:
#     print("All values match with MATLAB for quad_dynamics!")


# Print or inspect the results
# print("A Result:\n", A_result)
# print("\nB Result:\n", B_result)


FileNotFoundError: ignored

In [ ]:
def generate_dataset(X0, n_control, t_span, flag):
    # Set true and nominal system params
    params = {
        "mass": 0.3,
        "J": np.diag([1.43e-5, 1.43e-5, 2.89e-5]),
        "g": 9.81
    }
    params_nom = {
        "mass": params["mass"] * 2,
        "J": np.diag([(1.43e-5) * 1.5, (1.43e-5) * 1.5, (2.89e-5) * 1.2]),
        "g": params["g"]
    }

    # Generate random inputs for U
    mu = np.array([0, 0, 0, 0])
    if flag == "train":
        Sigma = np.diag([0.3, 5e-5, 5e-5, 5e-5])
    elif flag == "val":
        Sigma = np.diag([0.6, 5e-5, 5e-5, 5e-5])
    else:
        raise ValueError("Invalid flag value. Expected 'train' or 'val'.")
    U_rnd = np.random.multivariate_normal(mu, Sigma, n_control)
    U_rnd[:, 0] += params["mass"] * params["g"]

    # Initialize matrices for storing trajectories
    traj_len = len(t_span) - 1
    X1 = np.zeros((len(X0), traj_len * n_control))
    X2 = np.zeros((len(X0), traj_len * n_control))
    X1err = np.zeros((len(X0), traj_len * n_control))
    X2err = np.zeros((len(X0), traj_len * n_control))
    X1nom = np.zeros((len(X0), traj_len * n_control))
    X2nom = np.zeros((len(X0), traj_len * n_control))
    U = np.zeros((4, traj_len * n_control))

    # Main loop to generate trajectories
    for i in range(n_control):
        u = U_rnd[i, :].T

        # Simulate true trajectory
        x = odeint(lambda x, t: quad_dynamics(t, x, u, params), X0, t_span)

        # Generate one-step predictions using nominal model
        x_nominal = np.zeros((len(t_span), len(X0)))
        for k in range(len(t_span)):
            x_nominal[k, :] = odeint(lambda x, t: quad_dynamics(t, x, u, params_nom),  x[k, :], t_span[:2])[-1]

        # Organize the data
        X1[:, i*traj_len:(i+1)*traj_len] = x[:-1, :].T
        X2[:, i*traj_len:(i+1)*traj_len] = x[1:, :].T
        X1nom[:, i*traj_len:(i+1)*traj_len] = x_nominal[:-1, :].T
        X2nom[:, i*traj_len:(i+1)*traj_len] = x_nominal[1:, :].T
        X1err[:, i*traj_len:(i+1)*traj_len] = x[:-1, :].T - x_nominal[:-1, :].T
        X2err[:, i*traj_len:(i+1)*traj_len] = x[1:, :].T - x_nominal[1:, :].T
        U[:, i*traj_len:(i+1)*traj_len] = np.tile(u, (1, traj_len))

    return X1, X2, X1err, X2err, X1nom, X2nom, U


In [ ]:
# Author: KongYao Chee (ckongyao@seas.upenn.edu)
# Code adapted from: https://github.com/sriram-2502/KoopmanMPC_Quadrotor

import scipy.io

# Set random seed
np.random.seed(1)

# Options
training_method = 'pure_data'  # Options: 'pure_data', 'hybrid1', 'hybrid2'
# show_plot = False  # Plotting option (commented out as it's not available at the moment)

# Set initial parameters and conditions
dt = 1e-3
t_final = 0.1
t_span = np.arange(0, t_final + dt, dt)

# Initial state
p0 = np.array([0, 0, 0])
v0 = np.array([0, 0, 0])
R0 = np.eye(3)
w0 = np.array([0.1, 0, 0])
# Convert rotation matrix to quaternion using pyquaternion's Quaternion
q = Quaternion(matrix=R0.T).elements
x0 = np.concatenate([p0, v0, R0.ravel(), w0, q])
nx = len(x0)


## Generate new data
# Training dataset
# n_control = 100
# X1_train, X2_train, X1err_train, X2err_train, X1nom_train, X2nom_train, U_train = \
#     generate_dataset(x0, n_control, t_span, 'train')
# Here we initialize 3D arrays to store rotation matrices and then fill them with reshaped data
# R1_train = np.zeros((3, 3, X1_train.shape[1]))
# R2_train = np.zeros((3, 3, X2_train.shape[1]))
# R1nom_train = np.zeros((3, 3, X1nom_train.shape[1]))
# R2nom_train = np.zeros((3, 3, X2nom_train.shape[1]))

# for i in range(X1_train.shape[1]):
#     R1_train[:, :, i] = X1_train[6:15, i].reshape(3, 3)
#     R2_train[:, :, i] = X2_train[6:15, i].reshape(3, 3)
#     R1nom_train[:, :, i] = X1nom_train[6:15, i].reshape(3, 3)
#     R2nom_train[:, :, i] = X2nom_train[6:15, i].reshape(3, 3)
# Using pyquaternion to convert the rotation matrices to quaternions
# Using pyquaternion to convert the rotation matrices to quaternions
# Q1_train = [Quaternion(matrix=R1_train[:, :, i]).elements for i in range(R1_train.shape[2])]
# Q2_train = [Quaternion(matrix=R2_train[:, :, i]).elements for i in range(R2_train.shape[2])]
# Q1nom_train = [Quaternion(matrix=R1nom_train[:, :, i]).elements for i in range(R1nom_train.shape[2])]
# Q2nom_train = [Quaternion(matrix=R2nom_train[:, :, i]).elements for i in range(R2nom_train.shape[2])]


## Validation dataset
# n_control_val = 50
# X1_val, X2_val, X1err_val, X2err_val, X1nom_val, X2nom_val, U_val = \
#     generate_dataset(x0, n_control_val, t_span, 'val')
# Create dictionary for training data
# train_dat = {
#     "X1": X1_train,
#     "X2": X2_train,
#     "X1err": X1err_train,
#     "X2err": X2err_train,
#     "X1nom": X1nom_train,
#     "X2nom": X2nom_train,
#     "U": U_train
# }

# # Create dictionary for validation data
# val_dat = {
#     "X1": X1_val,
#     "X2": X2_val,
#     "X1err": X1err_val,
#     "X2err": X2err_val,
#     "X1nom": X1nom_val,
#     "X2nom": X2nom_val,
#     "U": U_val
# }

# # Save the dictionaries to .mat files
# scipy.io.savemat('train_dat_wquat.mat', train_dat)
# scipy.io.savemat('val_dat_wquat.mat', val_dat)


## Load training data from .mat file
dat_train = scipy.io.loadmat('train_dat_wquat.mat')
X1_train = dat_train["X1"]
X2_train = dat_train["X2"]
X1err_train = dat_train["X1err"]
X2err_train = dat_train["X2err"]
X1nom_train = dat_train["X1nom"]
X2nom_train = dat_train["X2nom"]
U_train = dat_train["U"]

# Load validation data from .mat file
dat_val = scipy.io.loadmat('val_dat_wquat.mat')
X1_val = dat_val["X1"]
X2_val = dat_val["X2"]
X1err_val = dat_val["X1err"]
X2err_val = dat_val["X2err"]
X1nom_val = dat_val["X1nom"]
X2nom_val = dat_val["X2nom"]
U_val = dat_val["U"]

import scipy.io

## Load Another dataset
# Uncomment the appropriate line depending on the dataset to load
# dat = scipy.io.loadmat('koopman_dat.mat')
# dat = scipy.io.loadmat('koopman_dat_err.mat')
# dat = scipy.io.loadmat('koopman_dat_err_wxyz.mat')

# # Extract training and validation data
# train_val_split_index = 5000

# X1_train = dat['data'][0:train_val_split_index-1, 1:23].T
# X2_train = dat['data'][1:train_val_split_index, 1:23].T
# X1nom_train = dat['data_nom'][0:train_val_split_index-1, 1:23].T
# X2nom_train = dat['data_nom'][1:train_val_split_index, 1:23].T
# X1err_train = dat['data_err'][0:train_val_split_index-1, 1:23].T
# X2err_train = dat['data_err'][1:train_val_split_index, 1:23].T
# U_train = dat['data'][0:train_val_split_index-1, 23:27].T

# X1_val = dat['data'][train_val_split_index:end-2, 1:23].T
# X2_val = dat['data'][train_val_split_index+1:end-1, 1:23].T
# X1nom_val = dat['data_nom'][train_val_split_index:end-2, 1:23].T
# X2nom_val = dat['data_nom'][train_val_split_index+1:end-1, 1:23].T
# X1err_val = dat['data_err'][train_val_split_index:end-2, 1:23].T
# X2err_val = dat['data_err'][train_val_split_index+1:end-1, 1:23].T
# U_val = dat['data'][train_val_split_index:end-2, 23:27].T

# Compute lengths of training and validation datasets
train_data_len = X1_train.shape[1]
val_data_len = X1_val.shape[1]

n_basis = 3
add_dim = 0

## Process training data based on the training method
if training_method == 'pure_data':
    Z1_train = np.zeros((nx + 9 * n_basis + add_dim, train_data_len))
    Z2_train = np.zeros((nx + 9 * n_basis + add_dim, train_data_len))

    for i in range(train_data_len):
        x1 = X1_train[:, i]
        x2 = X2_train[:, i]
        z1 = np.concatenate([x1[0:3].reshape(-1, 1), x1[3:6].reshape(-1, 1), construct_basis(x1, n_basis)], axis=0)
        z2 = np.concatenate([x2[0:3].reshape(-1, 1), x2[3:6].reshape(-1, 1), construct_basis(x2, n_basis)], axis=0)
        Z1_train[:, i] = z1.ravel()
        Z2_train[:, i] = z2.ravel()

elif training_method == 'hybrid1':
    Z1_train = np.zeros((nx + 9 * n_basis, train_data_len))
    Z2_train = np.zeros((nx + 9 * n_basis, train_data_len))
    Z1nom_train = np.zeros((nx + 9 * n_basis, train_data_len))
    Z2nom_train = np.zeros((nx + 9 * n_basis, train_data_len))

    for i in range(train_data_len):
        x1 = X1_train[:, i]
        x2 = X2_train[:, i]
        z1 = np.concatenate([x1[0:6], construct_basis(x1, n_basis)])
        z2 = np.concatenate([x2[0:6], construct_basis(x2, n_basis)])
        Z1_train[:, i] = z1
        Z2_train[:, i] = z2

        x1 = X1nom_train[:, i]
        x2 = X2nom_train[:, i]
        z1 = np.concatenate([x1[0:6], construct_basis(x1, n_basis)])
        z2 = np.concatenate([x2[0:6], construct_basis(x2, n_basis)])
        Z1nom_train[:, i] = z1
        Z2nom_train[:, i] = z2

elif training_method == 'hybrid2':
    Z1err_train = np.zeros((nx + 9 * n_basis, train_data_len))
    Z2err_train = np.zeros((nx + 9 * n_basis, train_data_len))

    for i in range(train_data_len):
        x1 = X1err_train[:, i]
        x2 = X2err_train[:, i]
        z1 = np.concatenate([x1[0:3], x1[3:6], construct_basis(x1, n_basis)])
        z2 = np.concatenate([x2[0:3], x2[3:6], construct_basis(x2, n_basis)])
        Z1err_train[:, i] = z1
        Z2err_train[:, i] = z2

# Training via EDMD
if training_method == 'pure_data':
    A, B = EDMD(Z1_train, Z2_train, U_train)

elif training_method == 'hybrid1':
    A, B = EDMD(Z1_train, Z2_train, U_train)
    A_nom, B_nom = EDMD(Z1nom_train, Z2nom_train, U_train)  # On the nominal states

elif training_method == 'hybrid2':
    A_err, B_err = EDMD(Z1err_train, Z2err_train, U_train)  # On the state errors

C = np.zeros((nx + add_dim, nx + 9 * n_basis + add_dim))
C[0:nx + add_dim, 0:nx + add_dim] = np.eye(nx + add_dim)

## Get prediction accuracy on training dataset
if training_method == 'pure_data':
    Z2_predicted = np.dot(A, Z1_train) + np.dot(B, U_train)

elif training_method == 'hybrid1':
    diff = np.dot(A, Z1_train) - np.dot(A_nom, Z1nom_train) + np.dot((B - B_nom), U_train)

elif training_method == 'hybrid2':
    Z2_error = np.dot(A_err, Z1err_train) + np.dot(B_err, U_train)

Xpred_train = np.zeros((nx + add_dim, train_data_len))
for i in range(train_data_len):
    if training_method == 'pure_data':
        Xpred_train[:, i] = np.dot(C, Z2_predicted[:, i])
    elif training_method == 'hybrid1':
        Xpred_train[:, i] = np.dot(C, diff[:, i]) + X2nom_train[:, i]
    elif training_method == 'hybrid2':
        Xpred_train[:, i] = np.dot(C, Z2_error[:, i]) + X2nom_train[:, i]


## Process validation dataset, to map states to basis vectors
if training_method == 'pure_data':
    Z1_val = np.zeros((nx + 9 * n_basis + add_dim, val_data_len))
    for i in range(val_data_len):
        x1 = X1_val[:, i]
        basis = construct_basis(x1, n_basis)
        z1 = np.concatenate([x1[:3].reshape(-1, 1), x1[3:6].reshape(-1, 1), basis], axis=0)
        Z1_val[:, i] = z1.ravel()

elif training_method == 'hybrid1':
    Z1_val = np.zeros((nx + 9 * n_basis, val_data_len))
    Z1nom_val = np.zeros((nx + 9 * n_basis, val_data_len))
    for i in range(val_data_len):
        x1 = X1_val[:, i]
        basis = construct_basis(x1, n_basis)
        z1 = np.concatenate([x1[:3], x1[3:6], basis])
        Z1_val[:, i] = z1

        x1 = X1nom_val[:, i]
        basis = construct_basis(x1, n_basis)
        z1 = np.concatenate([x1[:3], x1[3:6], basis])
        Z1nom_val[:, i] = z1

elif training_method == 'hybrid2':
    Z1err_val = np.zeros((nx + 9 * n_basis, val_data_len))
    for i in range(val_data_len):
        x1 = X1err_val[:, i]
        basis = construct_basis(x1, n_basis)
        z1 = np.concatenate([x1[:3], x1[3:6], basis])
        Z1err_val[:, i]

# Get prediction accuracy on validation dataset
if training_method == 'pure_data':
    Z2_predicted = np.dot(A, Z1_val) + np.dot(B, U_val)

elif training_method == 'hybrid1':
    diff = np.dot(A, Z1_val) - np.dot(A_nom, Z1nom_val) + np.dot((B - B_nom), U_val)

elif training_method == 'hybrid2':
    Z2_error = np.dot(A_err, Z1err_val) + np.dot(B_err, U_val)

Xpred_val = np.zeros((nx + add_dim, val_data_len))
for i in range(val_data_len):
    if training_method == 'pure_data':
        Xpred_val[:, i] = np.dot(C, Z2_predicted[:, i])

    elif training_method == 'hybrid1':
        Xpred_val[:, i] = np.dot(C, diff[:, i]) + X2nom_val[:, i]

    elif training_method == 'hybrid2':
        Xpred_val[:, i] = np.dot(C, Z2_error[:, i]) + X2nom_val[:, i]

## Get results
RMSE_training = compute_rmse(Xpred_train, X2_train)
RMSE_validation = compute_rmse(Xpred_val, X2_val)

print(RMSE_validation)


<ipython-input-2-abdd456dfe88>:5: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  matrix = np.array([[0, -vector[2], vector[1]],


(0.0025465558125148603, 0.002197871316090575, 0.011071608254926634, 0.006385569146884477, 0.36751993550173395)
